In [12]:
import os
import json

base_dir = "/mnt/drive-d/anurag/roadwork/"
det_data_dir = os.path.join(base_dir, "scene")

In [13]:
import mmdet.datasets
from mmdet.datasets import CocoDataset
from mmdet.registry import DATASETS

@DATASETS.register_module()
class WorkzoneInstanceDataset(CocoDataset):
    METAINFO = { 
        "classes" : (
        "Police Officer", # 1
        "Police Vehicle", # 2
        "Sidewalk", # 3
        "Cone", # 4
        "Fence", # 5
        "Drum", # 6
        "Barricade", # 7
        "Barrier", # 8
        "Work Vehicle", # 9
        "Vertical Panel", # 10
        "Tubular Marker", # 11
        "Arrow Board", # 12
        "Temporary Traffic Control Message Board", # 13
        "Road", # 14
        "Bike Lane", # 15
        "Temporary Traffic Control Sign", # 16
        "Work Equipment", # 17
        "Worker", # 18 
        "Other Roadwork Objects", # 19
        ),
        "palette" :
        [(220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228),
         (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30),
         (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30),
         (165, 42, 42), (255, 77, 255), (0, 226, 252), (182, 182, 255),
         (0, 82, 0), (120, 166, 157), (110, 76, 0), (174, 57, 255),
         (199, 100, 0), (72, 0, 118), (255, 179, 240), (0, 125, 92),
         (209, 0, 151), (188, 208, 182), (0, 220, 176), (255, 99, 164),
         (92, 0, 73), (133, 129, 255), (78, 180, 255), (0, 228, 0),
         (174, 255, 243), (45, 89, 255), (134, 134, 103), (145, 148, 174),
         (255, 208, 186), (197, 226, 255), (171, 134, 1), (109, 63, 54),
         (207, 138, 255), (151, 0, 95), (9, 80, 61), (84, 105, 51),
         (74, 65, 105), (166, 196, 102), (208, 195, 210), (255, 109, 65),
         (0, 143, 149), (179, 0, 194), (209, 99, 106), (5, 121, 0),
         (227, 255, 205), (147, 186, 208), (153, 69, 1), (3, 95, 161),
         (163, 255, 0), (119, 0, 170), (0, 182, 199), (0, 165, 120),
         (183, 130, 88), (95, 32, 0), (130, 114, 135), (110, 129, 133),
         (166, 74, 118), (219, 142, 185), (79, 210, 114), (178, 90, 62),
         (65, 70, 15), (127, 167, 115), (59, 105, 106), (142, 108, 45),
         (196, 172, 0), (95, 54, 80), (128, 76, 255), (201, 57, 1),
         (246, 0, 122), (191, 162, 208)]
    }

KeyError: 'WorkzoneInstanceDataset is already registered in dataset at __main__'

In [14]:
import cv2
from PIL import Image, ImageDraw
import numpy as np

PALLETE = WorkzoneInstanceDataset.METAINFO["palette"]
CLASSES = WorkzoneInstanceDataset.METAINFO["classes"]

def open_image(image_cv2):
    image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
    # Open image
    img_open = Image.fromarray(image_cv2).convert("RGBA")
    # Create layer for bboxes and masks
    draw_layer = Image.new("RGBA", img_open.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(draw_layer)
    return img_open, draw_layer, draw

def draw_objects(img_path, objects, categories, out_dir, alpha=127):
    img_cv2 = cv2.imread(img_path)
    img_open, draw_layer, draw = open_image(img_cv2)

    names_colors = [ (x['id'], x["name"], PALLETE[i]) for i, x in enumerate(categories) ]

    mask_objects = objects
    obj_colors = []
    for obj in mask_objects:
        cat_id = obj["category_id"]
        _, c = [ x[1:] for x in names_colors if x[0] == cat_id ][0]
        obj_colors.append(c)

    ## draw masks
    masks = [obj["segmentation"] for obj in mask_objects]
    for idx, (c, m) in enumerate(zip(obj_colors, masks)):
        alpha = alpha
        fill = tuple(list(c) + [alpha])
        outline = tuple(list(c) + [255])
        if isinstance(m, list):
            for m_ in m:
                try:
                    if m_:
                        draw.polygon(m_, outline=fill, fill=fill, width=5)
                except:
                    continue
        else:
            continue

    ## draw polylines
    poly_objects = objects
    poly_obj_colors = []
    for obj in poly_objects:
        cat_id = obj["category_id"]
        _, c = [ x[1:] for x in names_colors if x[0] == cat_id ][0]
        poly_obj_colors.append(c)
    try:
        polys = [ [ x["segmentation"], x["attributes"]["occluded"] ] for x in poly_objects]
    except:
        polys = [ [ x["segmentation"], False ] for x in poly_objects]
    for c, (poly, occluded) in zip(poly_obj_colors, polys):
        outline = tuple(list(c) + [255])
        try:
            for m in poly:
                if m:
                    draw.polygon(m, outline=outline, fill=None, width=2)
        except:
            continue

    composed_image = Image.alpha_composite(img_open, draw_layer)
    composed_image.save(os.path.join(out_dir, os.path.splitext(os.path.basename(img_path))[0] + ".png" ))

In [15]:
config = "./models/instance_gps_split/mask-rcnn_r50.py"
checkpoint = "./models/instance_gps_split/roadwork_segm_model_gps_split.pth"
device = "cuda:0"

## The json files are in the standard COCO format for instance segmentation
## however, they may contain additional fields like "attributes" for each object
## or "gps" or "scene_description" for each image. These fields are ignored by
## the COCO dataset loader.
val_data = os.path.join(det_data_dir, "annotations", "instances_val_gps_split.json")
with open(val_data, "r") as f:
    val_data = json.load(f)

im_info = val_data["images"][100]
img_path = os.path.join(det_data_dir, "images", im_info["file_name"])
objects = [ x for x in val_data["annotations"] if x["image_id"] == im_info["id"] ]
categories = val_data["categories"]

os.makedirs("./output/instance_gps_split/gt", exist_ok=True)
draw_objects(img_path, objects, categories, out_dir="./output/instance_gps_split/gt", alpha=170)

In [16]:
from mmdet.apis import DetInferencer

inferencer = DetInferencer(config, checkpoint, device)
result = inferencer(img_path, return_vis=False)

## rle to polygon
from pycocotools import mask as mask_utils
def mask_rle_to_polygon(rle):
    mask = mask_utils.decode(rle)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [c.flatten().tolist() for c in contours]

## convert MMDET result to COCO result format
mmdet_result = []
result = result['predictions'][0]
for label, scores, bbox, mask in zip(result['labels'], result['scores'], result['bboxes'], result['masks']):
    ## convert mmdet label to class id

    mmdet_result.append({
        "category_id" : label,
        "score" : scores,
        "bbox" : bbox,
        "segmentation" : mask_rle_to_polygon(mask)
    })

os.makedirs("./output/instance_gps_split/vis", exist_ok=True)
draw_objects(img_path, mmdet_result, categories, out_dir="./output/instance_gps_split/vis", alpha=170)

Loads checkpoint by local backend from path: ./models/instance_gps_split/roadwork_segm_model_gps_split.pth
